In [44]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.cross_validation import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import scipy.stats as stats
import seaborn as sns

%pwd
#os.chdir('C:\\Users\\Prudhvi\\Desktop\\Prudhvi\\Data Science\\Data')
os.chdir('C:\\Users\\KASTU1\\Desktop\\Analytics Path\\R\\Data')
%pwd

'C:\\Users\\KASTU1\\Desktop\\Analytics Path\\R\\Data'

In [88]:
churn=pd.read_csv("Teleco_Cust_Attr_1.csv",sep=',')

In [78]:
churn.shape

(7043, 21)

In [89]:
cm_train=confusion_matrix(churn['Churn'],churn['Churn'])
print(cm_train)

[[5174    0]
 [   0 1869]]


In [90]:
## Data Type Conversion
churn["SeniorCitizen"]=churn["SeniorCitizen"].astype(object)

## Replacing few values 
churn.loc[churn.OnlineSecurity=="No internet service","OnlineSecurity"]="No"
churn.loc[churn.OnlineBackup=="No internet service","OnlineBackup"]="No"
churn.loc[churn.DeviceProtection=="No internet service","DeviceProtection"]="No"
churn.loc[churn.TechSupport=="No internet service","TechSupport"]="No"
churn.loc[churn.StreamingTV=="No internet service","StreamingTV"]="No"
churn.loc[churn.StreamingMovies=="No internet service","StreamingMovies"]="No"
churn.loc[churn.MultipleLines=="No phone service","MultipleLines"]="No"

In [91]:
## Dropping few columns
churn.drop("customerID", axis = 1, inplace=True)
churn.drop("gender",axis=1,inplace=True)
#churn.drop("PhoneService",axis=1,inplace=True)

In [92]:
## Feature Engineering
churn.insert(0, "Charges", 0)
total_rows = churn['Churn'].count()
for i in range(0,total_rows):
    churn.loc[i,"Charges"]=churn.loc[i,"tenure"]*churn.loc[i,"MonthlyCharges"]

In [93]:
churn.drop("TotalCharges", axis = 1,  inplace=True)
churn.drop("tenure",axis=1,inplace=True)
churn.drop("MonthlyCharges",axis=1,inplace=True)

In [94]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()
for i in churn.columns:
    if(churn[i].dtype.name=="object"):
        churn[i] = churn[i].astype("category")
        churn[i] = lc.fit_transform(churn[i])
        churn[i] = churn[i].astype("category")
        
# master_dataset.head(7)
#churn.dtypes
churn['SeniorCitizen']=churn['SeniorCitizen'].astype('uint8')
churn['Partner']=churn['Partner'].astype('uint8')
churn['Dependents']=churn['Dependents'].astype('uint8')
churn['PaperlessBilling']=churn['PaperlessBilling'].astype('uint8')

In [95]:
cm_train=confusion_matrix(churn['Churn'],churn['Churn'])
print(cm_train)

[[5174    0]
 [   0 1869]]


In [96]:
from sklearn.preprocessing import OneHotEncoder
columns_labled=['Charges', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod']
oe = OneHotEncoder()
churn_cat=[]
churn_ncat=[]
for i in columns_labled:
    if(churn[i].dtype.name=="category"):
        if(len(churn[i].unique()) > 2):
            churn_cat.append(i) 
        else:
            churn_ncat.append(i)
    else:
        churn_ncat.append(i)
                 
churn_cat = churn[churn_cat]
churn_ncat = churn[churn_ncat]
fn_dataset = pd.merge(churn_cat,churn_ncat,left_index=True, right_index=True)


In [97]:
cm_train=confusion_matrix(churn['Churn'],churn['Churn'])
print(cm_train)

[[5174    0]
 [   0 1869]]


In [99]:
churn_cat = pd.get_dummies(churn_cat,drop_first=True)
X = fn_dataset.iloc[:,:-1]
Y = fn_dataset.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)

In [75]:
X.columns

Index(['InternetService_1', 'InternetService_2', 'Contract_1', 'Contract_2',
       'PaymentMethod_1', 'PaymentMethod_2', 'PaymentMethod_3', 'Charges',
       'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'],
      dtype='object')

In [76]:
cm_train=confusion_matrix(Y,Y)
print(cm_train)

[[2872    0]
 [   0 4171]]


In [100]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(penalty='l2',C=5)
print(classifier)
result=classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
coefficients = pd.concat([pd.DataFrame(X_train.columns),pd.DataFrame(np.transpose(result.coef_))], axis = 1)
print(coefficients)

print(result)
y_pred_train = result.predict(X_train)
y_pred_test = result.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(y_train,y_pred_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)

print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train)
print(classificaiton_report_train)

cm_test=confusion_matrix(y_test,y_pred_test)
print("Confusion Matrix of test Dataset")
print(cm_test)
TP=cm_test[1][1]
FP=cm_test[0][1]
FN=cm_test[1][0]
TN=cm_test[0][0]
precision_test=TP/(TP+FP)
Recall_test=TP/(TP+FN)
Accuracy_test=(TP+TN)/(TP+FP+TN+FN)
F1_Score_test=(2*precision_test*Recall_test)/(precision_test+Recall_test)
print("Precision of Test Data",+precision_test)
print("Rrecall of Test Data",+Recall_test)
print("Accuracy of Test Data",+Accuracy_test)
print("F1 Score of Test Data",+F1_Score_test)
classificaiton_report_TEST=classification_report(y_test,y_pred_test)
print(classificaiton_report_train)
#print("classification report of Train Dataset")
#classificaiton_report_test=classification_report(y_test,y_pred_test)
#print(classificaiton_report_test)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.6851571374479364
                   0         0
0    InternetService -0.203381
1           Contract -0.651127
2      PaymentMethod -0.047005
3            Charges  0.000141
4      SeniorCitizen  0.399471
5            Partner  0.022445
6         Dependents -0.154880
7       PhoneService  0.182505
8      MultipleLines  0.338135
9     OnlineSecurity -0.338064
10      OnlineBackup  0.151111
11  DeviceProtection -0.048127
12       TechSupport -0.119825
13       StreamingTV  0.476397
14   StreamingMovies  0.439444
LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001

C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
